# Faultproof Withdrawals Triage Runbook
In order to start this runbook locally please use the README located at [https://github.com/ethereum-optimism/monitorism/blob/main/op-monitorism/faultproof_withdrawals/runbooks/automated/README.md](https://github.com/ethereum-optimism/monitorism/blob/main/op-monitorism/faultproof_withdrawals/runbooks/automated/README.md)

General instructions to start the runbook locally:
```bash
cd op-monitorism/faultproof_withdrawals/runbooks/automated/
make start
```


In [1]:
from dotenv import load_dotenv
import os
from web3 import Web3
from lib.superchain import *
from lib.web3 import *
from pprint import pprint

#parameters setup (there should be no need to change the one below)
abi_folder_path="abi"


## setup 
In order to proceed make sure you set the parameters below.
The url used below are the trusted nodes that are going to be used.

In [2]:
#set ignore_certificate if you are using a local https
ignore_url_certificate=True

#set local nodes
L1_GETH_URL="https://proxyd-l1.rpc"
L2_OP_NODE_URL="https://op-node.rpc"
L2_OP_GETH_URL="https://op-geth.rpc"

#set cahin you need to monitor
l1_chain_name="mainnet"
l2_chain_name="op"

## Loading local values and superchain regsitry values

In [3]:
superchain=get_superchain_file(l1_chain_name, l2_chain_name)
OptimismPortalProxy=superchain["addresses"]["OptimismPortalProxy"]

print(f"OptimismPortal2 address: https://etherscan.io/address/{OptimismPortalProxy}#readProxyContract")

web3_utility=Web3Utility(L1_GETH_URL, L2_OP_GETH_URL,L2_OP_NODE_URL,abi_folder_path, OptimismPortalProxy, ignore_certificate=ignore_url_certificate)

OptimismPortal2 address: https://etherscan.io/address/0xbEb5Fc579115071764c7423A4f12eDde41f106Ed#readProxyContract


# Triage alert: faultproof-withdrawal-forgery-detected

An event is considered a forgery if any of the following conditions apply:
1. The withdrawalHash is not present on L2. We check this by querying [L2ToL1MessagePasser](https://github.com/ethereum-optimism/optimism/blob/develop/packages/contracts-bedrock/src/L2/L2CrossDomainMessenger.sol).
2. The [outputRoot provided](https://github.com/ethereum-optimism/optimism/blob/dd2b21ce786f4c1b722bda270348597182153c8e/packages/contracts-bedrock/src/L1/OptimismPortal2.sol#L314C15-L314C25) does not match what we see on [L2 block rootState](https://github.com/ethereum-optimism/monitorism/blob/c0b2ecdf4404888e5ceccf6ad14e35c5e5c52664/op-monitorism/faultproof_withdrawals/validator/op_node_helper.go#L47).

During the triage we need to make sure that the found withdrawalHash is really a malicious one.

Given a withdrawal event hash, that refers to this alert we are going to look for the corresponding game and verify the withdrawal is truly malicious.

In [5]:
#set up one of the following variable.
Txhash=""

#Txhash="0xf290e62898913197bc96033a29c098aeb26613d504cfa51bf5ca5ac90cca58da" Example of a Txhash


res=web3_utility.get_withdrawal_proven_extension_1(Txhash)
res=res[0]
transactionHash=res["transactionHash"].hex()
print(f"transactionHash: 0x{transactionHash}")
proofSubmitter=res["args"]["proofSubmitter"]
print(f"proofSubmitter: {proofSubmitter}")
withDrawalHash=res["args"]["withdrawalHash"].hex()
print(f"withdrawalHash: 0x{withDrawalHash}")

gameData=web3_utility.get_game_data(withDrawalHash,proofSubmitter)
print(f"gameData: {gameData}")

if gameData["rootClaim"] == gameData["optimism_outputAtBlock"]:
    print(f"rootClaim: {gameData['rootClaim']} == optimism_outputAtBlock: {gameData['optimism_outputAtBlock']}")
    print("game claim is valid")
else:
    print(f"rootClaim: {gameData['rootClaim']} != optimism_outputAtBlock: {gameData['optimism_outputAtBlock']}")
    print("game claim is invalid")

if gameData["sentMessages"] == True:
    print("sentMessages: True")
    print("withdrawal present on L2")



/Users/raffaele/projects/monitorism/utilities/runbooks/.venv/lib/python3.12/site-packages/eth_utils/functional.py:47: UserWarning: The log with transaction hash: HexBytes('0xf290e62898913197bc96033a29c098aeb26613d504cfa51bf5ca5ac90cca58da') and logIndex: 308 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  return callback(fn(*args, **kwargs))


transactionHash: 0xf290e62898913197bc96033a29c098aeb26613d504cfa51bf5ca5ac90cca58da
proofSubmitter: 0x440888714A6afeD60ff44e9975A96E6a36f7Fac4
withdrawalHash: 0x92c04040516e9be25714248e7d54defc4084d297335cb6604781df361dc9498f
gameData: {'gameProxyAddress': '0x093F8d13f85Ba25138AaDc0FaA0E0D0856A89455', 'timestamp': 1728581723, 'l2BlockNumber': 126489953, 'rootClaim': '0xf30bb800c4fdb4b210eaa665fd3e290026c9aa325d260d5ca4e1bff3f30e086e', 'sentMessages': True, 'optimism_outputAtBlock': '0xf30bb800c4fdb4b210eaa665fd3e290026c9aa325d260d5ca4e1bff3f30e086e'}
rootClaim: 0xf30bb800c4fdb4b210eaa665fd3e290026c9aa325d260d5ca4e1bff3f30e086e == optimism_outputAtBlock: 0xf30bb800c4fdb4b210eaa665fd3e290026c9aa325d260d5ca4e1bff3f30e086e
game claim is valid
sentMessages: True
withdrawal present on L2


# Triage alert: faultproof-withdrawal-forgery-detection-stalled
This alert monitors the number of withdrawal events that are considered normal in a chain. If the number of withdrawal events goes below a specified threshold, we trigger this alert.

We want to verify when it was last withdrawals on the chain. If there are no provenwithdrawals events on the chain in the last day, then we know the alert is a false positive and there is no issue with the monitor.

In [15]:
result=web3_utility.find_latest_withdrawal_event()

log=result["log"]
timestamp=result["timestamp"]

print(f"last withdrawal event:\n{timestamp["time_since_last_withdrawal"]} ago \n{timestamp["formatted_timestamp"]}")
print(f"last withdrawal happened: \nat block https://etherscan.io/block/{log["blockNumber"]} \nwith transaction hash https://etherscan.io/tx/0x{log["transactionHash"].hex()}")
print(f"check logs at: https://etherscan.io/address/{OptimismPortalProxy}#events using filter:0x798f9f13695f8f045aa5f80ed8efebb695f3c7fe65da381969f2f28bf3c60b97")

last withdrawal event:
1:27:58.831714 ago 
2024-10-10 17:35:23
last withdrawal happened: 
at block https://etherscan.io/block/20936742 
with transaction hash https://etherscan.io/tx/0xf290e62898913197bc96033a29c098aeb26613d504cfa51bf5ca5ac90cca58da
check logs at: https://etherscan.io/address/0xbEb5Fc579115071764c7423A4f12eDde41f106Ed#events using filter:0x798f9f13695f8f045aa5f80ed8efebb695f3c7fe65da381969f2f28bf3c60b97
